In [1]:
#! pip3 install tensorflow tensorflow-hub numpy 
import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import re
from qdrant_client import QdrantClient
from qdrant_client.http import models
import time

2024-03-09 16:00:05.912463: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 16:00:05.912484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 16:00:05.913086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-09 16:00:05.916801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 16:00:06.464763: W tensorflow/compiler/tf2

In [2]:
# Download the model and load the model

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)

In [3]:
# Function to generate embeddings

def embeddings(text):
    return np.array(model(text)).flatten().tolist()

In [4]:
# Dataset available at https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
# Download the dataset and extract the files before running this cell
# We will create the embeddings from this dataset using the universal-sentence-encoder model

file_path = 'SMSSpamCollection'

with open(file_path) as file:
    lines = [line for line in file]

msgs = [{"SMS":x.split('\t')[1].replace('\n', '')}   for x in lines]
embdngs = [embeddings([x["SMS"]]) for x in msgs]
indx = list(range(1, len(msgs)+1))



In [5]:
# Connect to Qdrant
client = QdrantClient("localhost", port=6333, timeout=60)

try:
    client.delete_collection(collection_name="SMS")
except:
    pass

In [6]:
# Collection with named Vectors

client.create_collection(
    collection_name="SMS",
    vectors_config={
        "txt_embedding": models.VectorParams(size=512, distance=models.Distance.EUCLID)
    },
)


True

In [7]:
# Batch insert
batch_size = 1000

id = [indx[i:i + batch_size] for i in range(0, len(indx), batch_size)]
payload = [msgs[i:i + batch_size] for i in range(0, len(msgs), batch_size)]
vector = [{"txt_embedding":embdngs[i:i + batch_size]} for i in range(0, len(embdngs), batch_size)]

for x in range(len(indx)//batch_size):

    client.upsert(
        collection_name="SMS",
        points=models.Batch(
            ids=id[x],
            payloads=payload[x],
            vectors=vector[x]),  wait=True
    )
    time.sleep(2)

In [8]:
# test message
test_message = ["claim prize"]
test_message_vector = embeddings(test_message)

In [9]:
## Vector Similarity Search
client.search(
    collection_name="SMS",
    query_vector=( "txt_embedding",test_message_vector),
    limit=5,with_vectors=False
)


[ScoredPoint(id=4048, version=4, score=1.0884774, payload={'SMS': 'Win a £1000 cash prize or a prize worth £5000'}, vector=None, shard_key=None),
 ScoredPoint(id=577, version=0, score=1.0934176, payload={'SMS': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}, vector=None, shard_key=None),
 ScoredPoint(id=4282, version=4, score=1.1073261, payload={'SMS': 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}, vector=None, shard_key=None),
 ScoredPoint(id=9, version=0, score=1.1073261, payload={'SMS': 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}, vector=None, shard_key=None),
 ScoredPoint(id=4037, version=4, score=1.1073704, payload={'SMS': 'YOU ARE CHOSEN TO RECEIVE A £350 AWARD! Pls call claim number 09066364311 to collect your award which you

In [10]:
# Search vectors along with payload

client.search(
    collection_name="SMS",
    query_vector=( "txt_embedding",test_message_vector),
    limit=5,
    query_filter=models.Filter(
    should=[
        models.FieldCondition(
            key="SMS",
            match=models.MatchText(text="million"),
            ),
            ]
),
)

[ScoredPoint(id=931, version=0, score=1.1438426, payload={'SMS': 'money!!! you r a lucky winner ! 2 claim your prize text money 2 88600 over £1million to give away ! ppt150x3+normal text rate box403 w1t1jy'}, vector=None, shard_key=None),
 ScoredPoint(id=4277, version=4, score=1.3449762, payload={'SMS': 'Haiyoh... Maybe your hamster was jealous of million'}, vector=None, shard_key=None)]